<img src="https://imgur.com/3U3hI1u.png" width="100%" />

> descrever aqui em que consiste a tarefa

## Escolha do Ambiente

Antes de analisar o possíveis algoritmos, o primeiro passo é escolher qual ambiente você quer resolver! Para esta tarefa, separamos quatro possíveis ambientes diferentes, em ordem de dificuldade, que você poderá escolher: **Pong**, **Lunar Lander**, **Lunar Lander Continuous** e **Car Racing**.

A seguir, estão as descrições de cada um deles:

<h2 align="center">Pong</h2>
<img src="https://imgur.com/vdVCmvo.gif" width=50% />

**Pong** é o ambiente de Aprendizado por Reforço criado pelo Turing que simula o jogo de *Pong*, no qual existem duas "raquetes" e uma bola, e o objetivo de cada uma das raquetes é não somente evitar que a bola passe por ela, como também fazer com que esta passe pela linha que a outra raquete protege.

### Características do Ambiente

O **Espaço de Observação** do ambiente é definido por 2 informações.

| Estado    | Informação                            |
| :-------- | :------------------------------------ |
| 0         | Distância _x_ entre a bola e o agente |
| 1         | Distância _y_ entre a bola e o agente |

Já o **Espaço de Ação** é composto por três ações: mover o jogador para cima, baixo, ou deixá-lo parado.

| Ação | Significado      |
| :--- | :--------------- |
| 0    | Ficar parado     |
| 1    | Mover para baixo |
| 2    | Mover para cima  |

Por fim, cada vez que tomamos uma ação, recebemos do ambiente uma **recompensa**, conforme a tabela abaixo:

| Ocorrência          | Recompensa |
| :------------------ | ---------: |
| Ponto do Agente     | $+500$     |
| Ponto do Oponente   | $-500$     |
| Vitória do Agente   | $+2000$    |
| Vitória do Oponente | $-2000$    |

O primeiro jogador a fazer quatro pontos ganha o jogo. Além disso, as recompensas são cumulativas. Isso significa que se o oponente fizer um ponto _e_ ganhar o jogo, a recompensa é de $-2500$.

### Instalação

Para instalar os ambientes criados pelo Turing, basta rodar o seguinte comando no terminal:

```cmd
!pip install -U turing-envs
```

Em seguida, para criar o ambiente, roda-se a linha de código a seguir:

```python
env = gym.make("turing_envs:pong-easy-v0")
```

<h2 align="center">Lunar Lander</h2>

<img src="https://raw.githubusercontent.com/fakemonk1/Reinforcement-Learning-Lunar_Lander/master/images/3.gif" width=50% />

**Lunar Lander** é um ambiente do Gym que simula o pouso de um módulo lunar na Lua. O agente deve controlar os três motores do módulo para guiá-lo até a pista de pouso, sem gastar muita energia.

### Características do Ambiente

O **Espaço de Observação** do ambiente é definido por 8 informações.

| Estado    | Informação                                     |
| :-------- | :--------------------------------------------- |
| 0         | Posição no eixo _x_ do módulo                  |
| 1         | Posição no eixo _y_ do módulo                  |
| 2         | Velocidade no eixo _x_ do módulo               |
| 3         | Velocidade no eixo _y_ do módulo               |
| 4         | Ângulo do módulo                               |
| 5         | Velocidade angular do módulo                   |
| 6         | Se a perna esquerda está em contato com o chão |
| 7         | Se a perna direita está em contato com o chão  |

Já o **Espaço de Ação** é composto por quatro ações: não fazer nada, acionar o motor esquerdo, acionar o motor principal ou acionar o motor direito.

| Ação | Significado             |
| :--- | :---------------------- |
| 0    | Não fazer nada          |
| 1    | Acionar motor esquerdo  |
| 2    | Acionar motor principal |
| 3    | Acionar motor direito   |

Por fim, cada vez que tomamos uma ação, recebemos do ambiente uma **recompensa**, conforme a tabela abaixo:

| Ocorrência              | Recompensa       |
| :---------------------- | ---------------: |
| Se aproximar da pista   | Até $+140$       |
| Pousar                  | $+100$           |
| Colidir                 | $-100$           |
| Tocar uma perna no chão | $+10$            |
| Acionar motor principal | $-0.3$ por frame |

#### Lunar Lander Continuous

Também existe uma versão contínua do ambiente do Lunar Lander, no qual podemos controlar a força que cada um dos motores do módulo exercerá. Nesse caso, teremos duas ações:

| Ação | Intervalo  | Significado                          |
| :--- | :--------: | :----------------------------------- |
| 0    | $-1$ a $1$ | Força do motor principal             |
| 1    | $-1$ a $1$ | Força dos motores esquerdo e direito |

Na versão contínua, os algoritmos que poderemos usar serão diferentes, e o treinamento provavlmente será mais difícil.

### Instalação

Para instalar os ambientes do Gym que usam a engine Box2D, é necessário rodar os seguintes comandos no terminal:


**Windows**
```cmd
!conda install swig
!pip install Box2D
!pip install pyglet==1.2.4
!pip install gym[box2d]
```

**Linux**
```bash
!apt install swig
!pip install Box2D
!pip install pyglet==1.2.4
!pip install gym[box2d]
```

Em seguida, para criar o ambiente, roda-se a linha de código a seguir:

```python
env = gym.make('LunarLander-v2')
```

ou

```python
env = gym.make('LunarLanderContinuous-v2')
```

<h2 align="center">Car Racing</h2>

<img src="https://camo.githubusercontent.com/10dd9fe65857382138418a936d23dace67c2f6d158b5dc0194c5219ef57d7c65/68747470733a2f2f6d656469612e67697068792e636f6d2f6d656469612f336f673049454b753834526f7339697a79552f67697068792e676966" width=50% />

**Car Racing** é um ambiente do Gym que simula um carro de corrida em uma pista. O agente deve pilotar o carro o mais rápido possível pela pista, até completar uma volta.

### Características do Ambiente

O **Espaço de Observação** do ambiente é uma imagem RGB de 96x96 pixels da vista superior do carro, como exemplificado a seguir:

<img src="https://imgur.com/7UFUngq.png" width=30% />

Já o **Espaço de Ação** é composto por três ações contínuas: o controle do volante, do acelerador e do freio.

| Ação | Intervalo  | Significado |
| :--- | :--------: | :---------- |
| 0    | $-1$ a $1$ | Volante     |
| 1    | $0$ a $1$  | Acelerador  |
| 2    | $0$ a $1$  | Freio       |

Por fim, cada vez que tomamos uma ação, recebemos do ambiente uma **recompensa**, conforme a tabela abaixo:

| Ocorrência               | Recompensa                                           |
| :----------------------- | ---------------------------------------------------: |
| Passar um bloco da pista | $\dfrac{+1000}{N}$ por bloco ($N$ = total de blocos) |
| A cada instante de tempo | $-0.1$                                               |

### Instalação

Para instalar os ambientes do Gym que usam a engine Box2D, é necessário rodar os seguintes comandos no terminal:


**Windows**
```cmd
!conda install swig
!pip install Box2D
!pip install pyglet==1.2.4
!pip install gym[box2d]
```

**Linux**
```bash
!apt install swig
!pip install Box2D
!pip install pyglet==1.2.4
!pip install gym[box2d]
```

Em seguida, para criar o ambiente, roda-se a linha de código a seguir:

```python
env = gym.make('CarRacing-v0')
```

## Criação do ambiente

### Escolha do Algoritmo

 - Por que escolheu o algoritmo?
   - Motivo pode ser baseado na teoria ou na prática.
 - Testou mais de um?

### Otimização de Hiperparâmetros

 - Interpretar as mudanças de _pelo menos_ 2 hiperparâmetros:
   - Sugestões: `gamma`, `learning_rate`.
   - Tente pensar e elaborar alguma teoria explicando os resultados encontrados.
 - [~~~Testar Policy Networks diferentes?~~~](https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html)